Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from typing import Tuple

Load and visualize dataset

In [ ]:
#Load data
train_data = pd.read_csv('Train.csv')

#Split data and labels
X = train_data.iloc[:, 1:]  
y = train_data.iloc[:, 0]

#Process data
if not isinstance(X, pd.DataFrame):
    X = pd.DataFrame(X)
X = X.apply(pd.to_numeric, errors='coerce')
X = X.fillna(0)  
X = X.values / 255.0
X = X.reshape(-1, 28, 28, 1)

#Visualize data as image on chosen index
plt.imshow(X[1].reshape(28, 28), cmap='gray')

ACTIVATION FUNCTION 


USE ReLU Functions to introduce non-linearity for model can do multi-class classification

In [ ]:
class ReLU:
    def __init__(self):
        self.cache = None 
    def forward(self, X: np.ndarray) -> np.ndarray:
        self.cache = X 
        # Luu nhung tham so duoc kich hoat de luc truyen GD thi chi truyen cho nhung tham so duoc kich hoat 
        return np.max(0,X)
    def backward(self, dout: np.ndarray) -> np.ndarray:
        X = self.cache
        return dout * (X > 0)

SOFTMAX

In [ ]:
class Softmax:
    def __init__(self):
        self.cache = None 
    def forward(self, X: np.ndarray) -> np.ndarray:
        exp_X = np.exp(X -np.max(X, axis=1, keepdims=True))
        self.cache = exp_X / np.sum(exp_X,axis =1, keepdims=True)
        return self.cache
    def backward(self, y: np.ndarray) -> np.ndarray:
        return self.cache - y
    def loss(self, y: np.ndarray) -> float:
        probs = self.cache
        #prevent log(0)
        probs = np.clip(probs, 1e-15, 1.0)
        n = probs.shape[0]
        loss = -cp.sum(y * np.log(probs) / n)
        return float(loss)

FULLY CONNECTED LAYER (FC)

In [ ]:
class Dense:
    def __init__(self, in_features: int, out_features: int):
        self.W = np.random.randn(in_features, out_features) * np.sqrt(2.0 / in_features)
        self.b = np.zeros((1, out_features))
        self.dW = None
        self.db = None
        self.cache = None
    
    def forward(self, X: np.ndarray) -> np.ndarray:
        self.cache = X
        return X @ self.W + self.b
    
    def backward(self, dout: np.ndarray) -> np.ndarray:
        X = self.cache
        self.dW = X.T @ dout
        self.db = cp.sum(dout, axis=0, keepdims=True)
        return dout @ self.W.T

BACKPROP 

OPTIMIZER

LEARNING AND TRAINING 

CONVOLUTION

PARAMETERS:
+ Image  : 3D ARRAY
+ K      : Kernel (or filters) small matrix slide on images pixel 
+ padding: The ammount pixel 
+ stride : jump of the kernel 

In [ ]:
class Conv2D:
    """Convolutional layer using im2col algorithm"""
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int = 1, padding: int = 0):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        # He initialization
        scale = np.sqrt(2.0 / (in_channels * kernel_size * kernel_size))
        self.W = np.random.randn(out_channels, in_channels, kernel_size, kernel_size) * scale
        self.b = np.zeros((out_channels, 1))
        
        self.dW = None
        self.db = None
        self.cache = None
    
    def forward(self, X: np.ndarray) -> np.ndarray:
        n, c, h, w = X.shape
        
        if self.padding > 0:
            X_padded = np.pad(X, ((0, 0), (0, 0), (self.padding, self.padding), (self.padding, self.padding)), mode='constant')
        else:
            X_padded = X
            
        h_out = (h + 2 * self.padding - self.kernel_size) // self.stride + 1
        w_out = (w + 2 * self.padding - self.kernel_size) // self.stride + 1
        
        # 1. Transform Image to Columns
        X_col = self.im2col(X_padded, self.kernel_size, self.stride)
        
        # 2. Reshape Weights: (Out_C, In_C * K * K)
        W_col = self.W.reshape(self.out_channels, -1)
        
        # 3. Matrix Multiplication: (Out_C, N * H_out * W_out)
        out = W_col @ X_col + self.b
        
        # 4. Reshape back to image format: (Out_C, H_out, W_out, N) -> (N, Out_C, H_out, W_out)
        out = out.reshape(self.out_channels, h_out, w_out, n)
        out = out.transpose(3, 0, 1, 2)
        
        self.cache = (X, X_col)
        return out
    
    def backward(self, dout: np.ndarray) -> np.ndarray:
        X, X_col = self.cache
        n, c, h, w = X.shape
        
        # Prepare dout: (Out_C, N * H_out * W_out)
        dout_reshaped = dout.transpose(1, 2, 3, 0).reshape(self.out_channels, -1)
        
        # Gradient w.r.t weights
        self.dW = (dout_reshaped @ X_col.T).reshape(self.W.shape)
        self.db = cp.sum(dout_reshaped, axis=1, keepdims=True)
        
        # Gradient w.r.t input column
        W_col = self.W.reshape(self.out_channels, -1)
        dX_col = W_col.T @ dout_reshaped
        
        # Column to Image
        dX = self.col2im(dX_col, X.shape, self.kernel_size, self.stride, self.padding)
        
        return dX
    
    def im2col(self, X: np.ndarray, kernel_size: int, stride: int) -> np.ndarray:
        """
        Transforms input image to column matrix.
        Output shape: (C * K * K, N * H_out * W_out)
        """
        n, c, h, w = X.shape
        h_out = (h - kernel_size) // stride + 1
        w_out = (w - kernel_size) // stride + 1
        
        # Index calculation
        i0 = np.repeat(cp.arange(kernel_size), kernel_size)
        i0 = np.tile(i0, c)
        i1 = stride * np.repeat(np.arange(h_out), w_out)
        
        j0 = np.tile(np.arange(kernel_size), kernel_size * c)
        j1 = stride * np.tile(np.arange(w_out), h_out)
        
        i = i0.reshape(-1, 1) + i1.reshape(1, -1)
        j = j0.reshape(-1, 1) + j1.reshape(1, -1)
        
        k = np.repeat(np.arange(c), kernel_size * kernel_size).reshape(-1, 1)
        
        # Get columns: (C*K*K, H_out*W_out, N) -> (C*K*K, N*H_out*W_out)
        # Using advanced indexing. This creates a large temporary array.
        cols = X[:, k, i, j] 
        cols = cols.transpose(1, 2, 0).reshape(c * kernel_size * kernel_size, -1)
        return cols
    
    def col2im(self, cols: np.ndarray, X_shape: Tuple, kernel_size: int, stride: int, padding: int) -> np.ndarray:
        n, c, h, w = X_shape
        h_padded, w_padded = h + 2 * padding, w + 2 * padding
        X_padded = np.zeros((n, c, h_padded, w_padded))
        
        h_out = (h_padded - kernel_size) // stride + 1
        w_out = (w_padded - kernel_size) // stride + 1
        
        i0 = np.repeat(np.arange(kernel_size), kernel_size)
        i0 = np.tile(i0, c)
        i1 = stride * np.repeat(np.arange(h_out), w_out)
        j0 = np.tile(np.arange(kernel_size), kernel_size * c)
        j1 = stride * np.tile(np.arange(w_out), h_out)
        i = i0.reshape(-1, 1) + i1.reshape(1, -1)
        j = j0.reshape(-1, 1) + j1.reshape(1, -1)
        k = np.repeat(np.arange(c), kernel_size * kernel_size).reshape(-1, 1)
        
        cols_reshaped = cols.reshape(c * kernel_size * kernel_size, -1, n)
        cols_reshaped = cols_reshaped.transpose(2, 0, 1)
        
        # Vectorized accumulation using add.at
        # Slice(None) corresponds to 'n' dimension
        np.add.at(X_padded, (slice(None), k, i, j), cols_reshaped)
        
        if padding > 0:
            return X_padded[:, :, padding:-padding, padding:-padding]
        return X_padded

POOLING

In [ ]:
def Max_Pooling(input_array, kernel_size=2, stride=1):
    k, s = kernel_size, stride
    
    # Calculate output shape
    output_shape = ((input_array.shape[0] - kernel_size) // stride) + 1
    
    # Initialize output array with correct shape
    output = np.zeros((output_shape, output_shape))
    
    # Perform max pooling
    for i in range(output_shape):
        for j in range(output_shape):
            # Extract the window
            window = input_array[i*s:i*s + k, j*s:j*s + k]
            # Get max value from window
            output[i][j] = np.max(window)
    
    return output



FLATTENING

In [ ]:
class Flatten:
    def __init__(self):
        self.cache = None
    def forward(self, X: np.ndarray) -> np.ndarray
        self.cache = X.shape
        retrun X.reshape(X.shape[0], -1)
    def backward(self, dout: np.ndarray) -> np.ndarray  
        return dout.reshape(self.cache)

SIGMOID AND TANH

BACKPROB AGAIN 

TESTING

MODEL LAYOUT
- Input : images (resolution of 28 x 28 and monochromatic) of hand written digits
- Block 1
    + Conv2D (1)
    + Activation (1)
    + Max Pooling (1)
- Block 2
    + Conv2D (2)
    + Activation (2)
    + Max Pooling (2)
- Flatten
- Block 3
    + Fully Connected (1)
    + Activation (3)
- Fully Connected
- Softmax